<a href="https://colab.research.google.com/github/nay-uku/studia/blob/main/diagnostyka-modelMM/dtu_projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>Temat:</b>
28. Projekt i implementacja algorytmu wyznaczającego optymalne struktury zgodne z modelem MM (2 osoby)


<b>Autorzy:</b>


*   Karol Baranowski WCY19KB2S4
*   Wojciech Grzyb   WCY19KB2S4



Załadowanie parametrów:

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving cube_3-true.txt to cube_3-true.txt
Saving cube_2-true.txt to cube_2-true.txt


In [ ]:
import re

# Załadowanie tabeli porównań
file_name = str(input("Podaj nazwę pliku z tabelą porównań: "))
comp_tests = []
with open(file_name, 'r') as f:
  for line in f:
    pattern = re.compile("^\|(\s+\d+\s+\|){4}$")
    if pattern.match(line):
      comp_test = [int(x) for x in re.findall("\d+", line)[1:]]  # Test porównawczy
      # Weryfikacja testu porównawczego:
      if len(comp_test) > len(set(comp_test)):
        raise Exception("Podana tabela porównań jest nieprawidłowa!")
      comp_tests.append(comp_test)
  if len(comp_tests) == 0:
    raise Exception("Nieprawidłowa struktura pliku!")

# Załadowanie parametru t
t = int(input("Podaj parametr t: "))


Podaj nazwę pliku z tabelą porównań: cube_3-true.txt
Podaj parametr t: 3


Konwersja tabeli porównań do sześcianu:

In [ ]:
import numpy as np
import sys

np.set_printoptions(threshold=sys.maxsize)

def comp_tests_to_cube(comp_tests):
  # Ustawienie zmiennych
  n_E = max(max(comp_tests))  # wyznaczenie rzędu grafu
  cube = np.zeros((n_E,n_E,n_E), int)  # deklaracja sześcianu
  for comp_test in comp_tests:
    e_k = comp_test[0]-1  # komparator
    e_i = comp_test[1]-1  # element porównawczy 1
    e_j = comp_test[2]-1  # element porównawczy 2
    cube[e_k,e_i,e_j] = 1
    cube[e_k,e_j,e_i] = 1
  return cube

cube = comp_tests_to_cube(comp_tests)

Implementacja funkcji sprawdzających czy graf jest t-diagnozowalny

In [ ]:
import itertools as it

def check_necessary_conditions(cube, t, debug=False):
  # Ustawienie zmiennych
  n_E = len(cube)  # rząd grafu
  # Weryfikacja warunku 1:
  if not (n_E >= 2*t +1):
    if debug:
      print("WK: warunek 1 nie jest spełniony!")
    return False
  # Weryfikacja warunku 2:
  for i in range(n_E):
    n_adj = len([j for j in range(n_E) if np.count_nonzero(cube[i][j] == 1) > 0])
    if not (n_adj >= t):
      if debug:
        print("WK: warunek 2 nie jest spełniony!")
      return False
  # WK są spełnione
  if debug:
    print("WK są spełnione.")
  return True


def get_comp_tests(cube):
  # Ustawienie zmiennych
  n_E = len(cube)  # rząd grafu
  comp_tests = []  # deklaracjia PSI'
  for e_k in range(n_E):
    for e_i in range(n_E):
      for e_j in range(n_E):
        if cube[e_k, e_i, e_j] == 1:
          comp_test = (e_k, (e_i, e_j))  # odczytanie testu porównawczego
          comp_tests.append(comp_test)  # dodanie odczytanego testu do zbioru testów porównawczych
  return comp_tests
  
def get_T_E1(cube, E1):
  # Ustawienie zmiennych
  T_E1 = set()  # deklaracja T(E`)
  comp_tests = get_comp_tests(cube)  # PSI
  for comp_test in comp_tests:
    e_k = comp_test[0]  # komparator
    e_i = comp_test[1][0]  # element porównawczy 1
    e_j = comp_test[1][1]  # element porównawczy 2
    if e_k in E1 and e_i in E1 and e_j not in E1:
      T_E1.add(e_j)
  return T_E1

def get_Gamma_E1(cube, E1):
  # Ustawienie zmiennych
  n_E = len(cube)  # rząd grafu
  Gamma_E1 = set()  # deklaracja Γ(E`)
  for e_k in E1:
    e_k_adj = [e_i for e_i in range(n_E) if np.count_nonzero(cube[e_k][e_i] == 1) > 0]
    for e_i in e_k_adj:
      if e_i not in E1:
        Gamma_E1.add(e_i)
  return Gamma_E1

def check_sufficient_condition(cube, t, debug=False):
  # Ustawienie zmiennych
  n_E = len(cube)  # rząd grafu
  E = [i for i in range(n_E)]  # otagowanie wierzchołków
  # Weryfikacja warunku:
  for s in range(t-1):
    n_E1 = n_E-2*t+s  # rząd Grafu E`
    # n_E1 = 2*(t-s)  # rząd Grafu E`
    all_E1 = it.combinations(E, n_E1)  # wygenerowanie wszystkich możliwych zbiorów E`
    for E1 in all_E1:
      T_E1 = get_T_E1(cube, E1)
      if not (len(T_E1) > s):
        if debug:
          print("WW: warunek nie jest spełniony!")
          print(f"E1: {E1}")
          print(f"T_E1: {T_E1}")
          print(f"len(T_E1): {len(T_E1)}")
          print(f"s: {s}")
        return False
  # WW są spełnione
  if debug:
    print("WW są spełnione.")
  return True

Weryfikacja czy wczytany sześcian jest prawidłowy:

In [ ]:
def is_cube_valid(cube, t, debug=False):
  # Sprawdzenie WK
  if not check_necessary_conditions(cube, t, debug):
    return False
  # Sprawdzenie WW
  if not check_sufficient_condition(cube, t, debug):
    return False
  # Sześciań jest prawidłowy
  if debug:
    print("Sześcian jest prawidłowy!")
  return True

is_cube_valid(cube, t, debug=True)

WK są spełnione.
WW: warunek nie jest spełniony!
E1: (0,)
T_E1: set()
len(T_E1): 0
s: 0


False

Generowanie optymalnej t-diagnozowalnej SOD:

In [ ]:
import math

def gen_opt_cube_naive(n, t):
  cube = np.zeros((n,n,n), int)  # deklaracja sześcianu
  # Wygenerowanie t+1 prób porównawczych dla każdego komparatora
  for i in range(n):
    arange = [x for x in range(n*n) if (x > math.floor(x/n)*(n+1)) and (i*n > x or x >= i*n+n) and (x%n != i) and (x%(n+1) != 0)]  # e_k != e_i and e_k != e_j and e_i != e_j
    if len(arange) < t+1:
      raise Exception("Podano niewłaściwe parametry!")
    for idx in np.random.permutation(arange)[:t+1]:
      row = math.floor(idx/n)
      col = idx - n*row
      cube[i, row, col] = 1
      cube[i, col, row] = 1
  return cube

def gen_opt_cube_analytic(t):
  n = 2*t+1
  cube = np.zeros((n,n,n), int)  # deklaracja sześcianu
  # Wygenerowanie t+1 prób porównawczych dla każdego komparatora
  for e_k in range(n):
    e_i = (e_k+1)%n
    for e_j in [e_j for e_j in range(n) if e_j != e_i and e_j != e_k and e_j != (e_k+2)%n ]:
      cube[e_k, e_i, e_j] = 1
      cube[e_k, e_j, e_i] = 1
  return cube

# Pomysł na optymalizację:
# Wykorzystać dynamic programming do równomiernego wylosowania prób porównawczych (line 35: e_j)
def gen_opt_cube_standard(n, t):
  cube = np.zeros((n,n,n), int)  # deklaracja sześcianu
  # Wygenerowanie t+1 prób porównawczych dla każdego komparatora
  for e_k in range(n):
    e_i = (e_k+1)%n
    for e_j in np.random.permutation([e_j for e_j in range(n) if e_j != e_i and e_j != e_k])[:t+1]:
      cube[e_k, e_i, e_j] = 1
      cube[e_k, e_j, e_i] = 1
  return cube

In [ ]:
t=3
cube = gen_opt_cube_analytic(t)
if check_necessary_conditions(cube, t, debug=True) and check_sufficient_condition(cube, t, debug=True):
  print(f"Graf jest {t}-diagnozowalny")
  print(cube)
else:
  print(f"Graf NIE jest {t}-diagnozowalny")

WK są spełnione.
WW: warunek nie jest spełniony!
E1: (0,)
T_E1: set()
len(T_E1): 0
s: 0
Graf NIE jest 3-diagnozowalny


In [ ]:
t=3
cube = gen_opt_cube_standard(2*t+2, t)
if check_necessary_conditions(cube, t, debug=True) and check_sufficient_condition(cube, t, debug=True):
  print(f"Graf jest {t}-diagnozowalny")
  print(cube)
else:
  print(f"Graf NIE jest {t}-diagnozowalny")

WK są spełnione.
WW: warunek nie jest spełniony!
E1: (1, 3)
T_E1: set()
len(T_E1): 0
s: 0
Graf NIE jest 3-diagnozowalny


In [ ]:
t=3
for i in range(100):
  cube = gen_opt_cube_naive(2*t+1, t)
  if i%5 == 0:
    print(f"i: {i}")
  if check_necessary_conditions(cube, t, debug=False):
    print(f"i: {i}; WK są spełnione.")
    if check_sufficient_condition(cube, t, debug=False):
      print(f"i: {i}; WK są spełnione.")
      print(f"Graf jest {t}-diagnozowalny")
      print(cube)
      break

i: 0
i: 0; WK są spełnione.
i: 1; WK są spełnione.
i: 2; WK są spełnione.
i: 3; WK są spełnione.
i: 4; WK są spełnione.
i: 5
i: 5; WK są spełnione.
i: 6; WK są spełnione.
i: 7; WK są spełnione.
i: 8; WK są spełnione.
i: 9; WK są spełnione.
i: 10
i: 10; WK są spełnione.
i: 11; WK są spełnione.
i: 12; WK są spełnione.
i: 13; WK są spełnione.
i: 14; WK są spełnione.
i: 15
i: 15; WK są spełnione.
i: 16; WK są spełnione.
i: 17; WK są spełnione.
i: 18; WK są spełnione.
i: 19; WK są spełnione.
i: 20
i: 20; WK są spełnione.
i: 21; WK są spełnione.
i: 22; WK są spełnione.
i: 23; WK są spełnione.
i: 24; WK są spełnione.
i: 25
i: 25; WK są spełnione.
i: 26; WK są spełnione.
i: 27; WK są spełnione.
i: 28; WK są spełnione.
i: 29; WK są spełnione.
i: 30
i: 30; WK są spełnione.
i: 31; WK są spełnione.
i: 32; WK są spełnione.
i: 33; WK są spełnione.
i: 34; WK są spełnione.
i: 35
i: 35; WK są spełnione.
i: 36; WK są spełnione.
i: 37; WK są spełnione.
i: 38; WK są spełnione.
i: 39; WK są spełnione.
i: 4

In [ ]:
def tran_matrix_to_cube(tran_matrix):
  # Ustawienie zmiennych
  n = len(tran_matrix)
  E = [i for i in range(n)]  # otagowanie wierzchołków
  n_E = len(E)
  # Określenie zbioru prób porównawczych
  all_Psi = []
  for e in range(n_E):
    all_adj_nodes = it.combinations(np.delete(E, e), 2)  # lista możliwych wierzchołków przyległych do komparatora e
    for adj_nodes in all_adj_nodes:
      if tran_matrix[e][adj_nodes[0]] == 1 and tran_matrix[e][adj_nodes[1]] == 1:
        all_Psi.append([e,adj_nodes])
  for i in range(n):
    all_adj_nodes = it.combinations(np.delete(E, e), 2)
    for adj_nodes in all_adj_nodes:
      if tran_matrix[e][adj_nodes[0]] == 1 and tran_matrix[e][adj_nodes[1]] == 1:
        all_Psi.append([e,adj_nodes])
  # Utworzenie sześcianu
  cube = np.zeros((n,n,n), int)  # deklaracja sześcianu
  for psi in all_Psi:
    e_k = psi[0]
    e_i = psi[1][0]
    e_j = psi[1][1]
    cube[e_k, e_i, e_j] = 1
    cube[e_k, e_j, e_i] = 1
  return cube

def cube_to_tran_matrix(cube):
  # Ustawienie zmiennych
  n = len(cube)
  tran_matrix = np.zeros((n,n), int)  # deklaracja macierzy przejść
  for k in range(n):
    for i in range(n):
      for j in range(n):
        if cube[k, i, j] == 1:
          tran_matrix[k, i] = 1
          tran_matrix[k, j] = 1
  return tran_matrix

In [ ]:
print(cube)
print('-----------------------')
print(cube_to_tran_matrix(cube))
print('-----------------------')
print(tran_matrix_to_cube(cube_to_tran_matrix(cube)))

[[[0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0]
  [0 0 0 1 1 0 1]
  [0 1 1 0 0 0 0]
  [0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 0 0 0 1 0 1]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0]
  [1 0 0 0 0 1 0]
  [0 0 0 1 1 0 0]
  [1 0 0 0 0 0 0]]

 [[0 0 0 1 1 0 1]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [1 0 0 0 0 1 0]
  [1 0 0 0 0 0 0]
  [0 0 0 1 0 0 0]
  [1 0 0 0 0 0 0]]

 [[0 1 0 0 1 0 0]
  [1 0 1 0 0 0 0]
  [0 1 0 0 0 0 1]
  [0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 0 1 1 0 0 0]
  [0 0 1 0 0 0 0]
  [1 1 0 0 0 1 0]
  [1 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0]]

 [[0 1 1 0 0 0 0]
  [1 0 0 0 0 0 1]
  [1 0 0 1 0 0 0]
  [0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 1 1 1 0]
  [0 0 1 0 0 0 0]
  [1 0 1 0 0 0 0]
  [0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0]]]
-----------------------
[[0 1 1 1 1 0 1]
 [1 0 0 1 1 1 1]
 [1 0 0 1 1 1 1]
 [1 1 1 0 1 0 1]
 [1 1 1 1 0 

In [ ]:
def print_uniq_comp_tests(cube):
  uniq_comp_tests = set()
  for test in get_comp_tests(cube):
    uniq_comp_tests.add((test[0]+1, tuple(sorted((test[1][0]+1,test[1][1]+1)))))
  print(f"{'P':<4} {'K':<2} {'u':<2} {'v':<2}")
  for i, test in enumerate(sorted(uniq_comp_tests)):
    print(f"{str(i+1)+'.':<4} {test[0]:<2} {test[1][0]:<2} {test[1][1]:<2}")

print_uniq_comp_tests(cube)

P    K  u  v 
1.   1  2  4 
2.   1  2  5 
3.   1  2  6 
4.   1  2  7 
5.   2  1  3 
6.   2  3  5 
7.   2  3  6 
8.   2  3  7 
9.   3  1  4 
10.  3  2  4 
11.  3  4  6 
12.  3  4  7 
13.  4  1  5 
14.  4  2  5 
15.  4  3  5 
16.  4  5  7 
17.  5  1  6 
18.  5  2  6 
19.  5  3  6 
20.  5  4  6 
21.  6  2  7 
22.  6  3  7 
23.  6  4  7 
24.  6  5  7 
25.  7  1  3 
26.  7  1  4 
27.  7  1  5 
28.  7  1  6 
